# [Work in progress] Load Hofree original datasets
[Network Based Stratification](http://chianti.ucsd.edu/~mhofree/wordpress/?page_id=26)

In [ ]:
from scipy.io import loadmat

In [0]:
tmp=!pwd
root=tmp[0]

In [0]:
dataFolder=root[:root.find('/Dropbox')]+'/Dropbox/python/stratipy/nbs_release_v0.2/data/'
data = loadmat(dataFolder+'TCGA_somatic_mutations/somatic_data_OV.mat')
network = loadmat(dataFolder+'networks/adj_mat.mat')
entrez_to_idmat = loadmat(dataFolder+'networks/entrez_to_idmat.mat')

In [0]:
print data.keys()
len(data['gene_id_all'])

In [0]:
mutations=data['gene_indiv_mat']
mutations.shape

In [0]:
print network.keys()
net=network['adj_mat']
net.shape

In [0]:
entrez_to_idmat.keys()

In [0]:
len(entrez_to_idmat['keymat'][0])

In [0]:
keys=[x[0] for x in entrez_to_idmat['keymat'][0]]
ids=[x[0][0] for x in entrez_to_idmat['entrezid'][0]]

In [0]:
entrez_to_idmat['keymat'][0][0][0]

In [0]:
keys[249]

In [0]:
ids[249]

In [0]:
genes = [x[0] for x in data['gene_id_all']]

In [0]:
genes

In [0]:
import numpy as np
l=[]
subnet=[]
good=[]
for j,g in enumerate(genes):
    try:
        i=ids.index(g)
        subnet.append(i)
        good.append(j)
    except:
        i=np.nan
    l.append(i)

In [0]:
l

In [0]:
subnet

In [0]:
nnet=net[subnet][:,subnet]
nnet.shape

In [0]:
nmut=mutations[:,good]
nmut.shape

In [0]:
import matplotlib as mpl
import matplotlib.pyplot as plt
plt.imshow(nmut)
plt.show()

In [0]:
type(np.matrix(nmut)),np.matrix(nmut).shape

In [0]:
np.matrix([val(g) for g in P]).shape

In [0]:
print type(A), A.shape

In [0]:
model = ProjectedGradientNMF(n_components=6, init='random', random_state=0)
model.fit(np.matrix(nmut))
sklearnComp=model.components_
plt.figure(1,figsize=(16,10))
plot(sklearnComp.T)
plt.show()

In [0]:
W,stratipyCompG = GNMF(np.matrix(nmut),np.matrix(nnet.todense()),0.2,n_components=3,tol=1e-3,max_iter=1000)

In [0]:
plt.figure(1,figsize=(16,10))
plot(stratipyCompG.T)
plt.show()

In [0]:
plt.figure(1,figsize=(16,10))
pos=nx.graphviz_layout(G,prog="neato")
for cn in range(0,6):
    node_color=array(np.round(stratipyCompG[cn]*100))
    plt.subplot(2, 3, cn+1)
    nx.draw(G,pos,with_labels=False,node_size=50,node_color=node_color,cmap = plt.cm.Blues)
plt.show()